In [38]:
import pandas as pd
import numpy as np
import altair as alt
import eco_style
alt.themes.enable('light')
import pycountry

In [2]:
df = pd.read_csv("forces.csv")
df['value'] = df['value'].str.replace(' ', '').astype(int)
df['label'] = np.where(df['date'] == df['date'].max(), 'Personnel', '')
df['date'] = pd.to_datetime(df['date'])

base = alt.Chart(df).encode(
    x=alt.X('date:T',
            axis=alt.Axis(labelFontSize=12, titleFontSize=12),
            ),
    y=alt.Y('value:Q', 
                        axis=alt.Axis(labelFontSize=12, titleFontSize=12),
            title=None),
)

line = base.mark_line(color="#36B7B4")

end_label = base.mark_text(
    align='left',
    color='#36B7B4',
    baseline='middle',
    dx=7
).encode(
    text='label'
)

chart = (line + end_label).properties(
    width=400,
    height=200,
)

# annual change df, jan to jan
a_df = df[df['date'].dt.month == 1].copy()
a_df['change'] = a_df['value'] - a_df['value'].shift(1)
a_df['year'] = a_df['date'].dt.strftime('%Y')

# add 0 vals for the first two years
extra = pd.DataFrame([
    {'date': pd.to_datetime('2013-01-01'), 'change': 0, 'label': '', 'year': '2013'},
])

a_df = pd.concat([
    extra,
    a_df
])


change_chart = alt.Chart(a_df).mark_bar(size=30, xOffset=0).encode(
    x=alt.X('year:N', 
            axis=alt.Axis(labelFontSize=12,
                          titleFontSize=12),
            title=None),
    y=alt.Y('change:Q',
            axis=alt.Axis(labelFontSize=12, titleFontSize=12),
             title=None),
    color=alt.condition(
        alt.datum.change > 0,
        alt.value('#36b7b4'),
        alt.value('#e6224b')
    ),
).properties(
    width=400,
    height=100,
)

change_chart





chart = alt.vconcat(chart, change_chart).resolve_scale(y='independent')

chart = chart.properties(
    title=alt.TitleParams(
        "UK Armed Forces Personnel",
        subtitle=["Full time trained strength, level and change", "Source: Gov.uk", ""],
        anchor='start',
        fontSize=16,
        subtitleFontSize=12,
    )
)

chart.save("chart.json")
chart.save("chart.png", scale_factor=3.0)

chart

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/

alt.VConcatChart(...)

# NATO Military Expenditure

In [53]:
df = pd.read_excel("nato_expend.xlsx", sheet_name="TABLE3", skiprows=7)
df = df.iloc[2:,2:]
df.columns = ['country'] + list(df.columns[1:])
df = df.head(30)

df.country = df.country.str.replace('*', '')
df = df.melt(id_vars='country', var_name='date', value_name='value')
df['date'] = df.date.astype(str).str.replace('e', '')
df['date'] = pd.to_numeric(df['date'], errors='coerce')
df['value'] = df.value/100
df['iso3'] = df['country'].apply(lambda x: pycountry.countries.get(name=x).alpha_3 if pycountry.countries.get(name=x) else None)

manual = {
    'Slovak Republic': 'SVK',
    'Türkiye': 'TUR',
}

df['iso3'] = df['iso3'].fillna(df['country'].map(manual))

df['flag'] = df['iso3'].apply(lambda x: pycountry.countries.get(alpha_3=x).flag if pycountry.countries.get(alpha_3=x) else None)

full_df = df.copy()


## Option 1

In [110]:
df = full_df.copy()
slider = alt.binding_range(min=2014, max=2023, step=1, name='Year:')
year_param = alt.param(bind=slider, value=2014, name='year_param')

base = alt.Chart(df).add_params(year_param).transform_filter(
    alt.datum.date == year_param
).encode(
    x=alt.X('iso3:N', axis=alt.Axis(labelFontSize=10, titleFontSize=12), sort='-y'),
    y=alt.Y('value:Q', axis=alt.Axis(labelFontSize=12, titleFontSize=12, format='%'), scale=alt.Scale(domain=[0,0.04]), title=None),
    color=alt.condition(
        alt.datum.iso3 == 'GBR',
        alt.value('#e6224b'),
        alt.value('#122b39')
)
)

bars = base.mark_bar()

two_pc_line = alt.Chart(pd.DataFrame({'y': [0.02]})).mark_rule(
    strokeDash=[3,3],
    opacity=0.5,
    color='#676A86').encode(y='y')

flags = base.mark_text(
    align='center',
    baseline='middle',
    dy=-7,
    dx=1,
    fontSize=16,
    color='black',
).encode(
    text='flag',
)

year_indicator = alt.Chart(pd.DataFrame({'foo': [1]})).add_params(year_param).transform_calculate(
    year="year_param"
).mark_text(
    align='center',
    baseline='top',
    fontSize=72,
    color='#122b39',
    opacity=1,
    text=alt.expr("year_param"),
).encode(
    x=alt.value(350),
    y=alt.value(100)
)

chart = two_pc_line + bars + flags  + year_indicator

chart = chart.properties(
    width=700,
    height=350,
    title=alt.TitleParams(
        "NATO Defence Spending",
        subtitle=["% of GDP, 2% target indicated", "Source: NATO", ""],
        subtitleColor='#676A86',
        anchor='start',
        fontSize=16,
        dx=35,
        subtitleFontSize=12,
    )
)

chart.save("nato_expend.json")
chart.save("nato_expend.png", scale_factor=3.0)

chart





/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/

alt.LayerChart(...)

In [112]:
df.query("date == 2023")

,country,date,value,iso3,flag
270,Albania,2023,0.017555,ALB,🇦🇱
271,Belgium,2023,0.011338,BEL,🇧🇪
272,Bulgaria,2023,0.018408,BGR,🇧🇬
273,Canada,2023,0.013822,CAN,🇨🇦
274,Croatia,2023,0.017949,HRV,🇭🇷
275,Czechia,2023,0.015050,CZE,🇨🇿
276,Denmark,2023,0.016467,DNK,🇩🇰
277,Estonia,2023,0.027266,EST,🇪🇪
278,Finland,2023,0.024497,FIN,🇫🇮
279,France,2023,0.019044,FRA,🇫🇷


In [96]:
year_param.name

'param_47'

## Option 2

In [61]:
ranks

,index,country,date,value,iso3,flag
0,291,Poland,2023,0.039011,POL,🇵🇱
1,299,United States,2023,0.034929,USA,🇺🇸
2,281,Greece,2023,0.030074,GRC,🇬🇷
3,277,Estonia,2023,0.027266,EST,🇪🇪
4,285,Lithuania,2023,0.025437,LTU,🇱🇹
5,278,Finland,2023,0.024497,FIN,🇫🇮
6,293,Romania,2023,0.024406,ROU,🇷🇴
7,282,Hungary,2023,0.024268,HUN,🇭🇺
8,284,Latvia,2023,0.022660,LVA,🇱🇻
9,298,United Kingdom,2023,0.020666,GBR,🇬🇧


In [76]:
df = full_df.copy()

# rank according to 2023
ranks = df[df['date'] == 2014].sort_values('value', ascending=False).reset_index(drop=True).reset_index().rename(columns={'index': 'rank'})[['iso3', 'rank']]
df = df.merge(ranks, on='iso3')

slider = alt.binding_range(min=2014, max=2023, step=1, name='Year:')
year_param = alt.param(bind=slider, value=2014)

base = alt.Chart(df).add_params(year_param).transform_filter(
    alt.datum.date == year_param
).encode(
    x=alt.X('iso3:N', axis=alt.Axis(labelFontSize=10, titleFontSize=12), sort=alt.SortField('rank')),
    y=alt.Y('value:Q', axis=alt.Axis(labelFontSize=12, titleFontSize=12, format='%'), scale=alt.Scale(domain=[0,0.04]), title=None),
    color=alt.condition(
        alt.datum.iso3 == 'GBR',
        alt.value('#e6224b'),
        alt.value('#122b39')
)
)

bars = base.mark_bar()

flags = base.mark_text(
    align='center',
    baseline='middle',
    dy=-7,
    fontSize=16,
    color='black',
).encode(
    text='flag',
)

chart = bars + flags

chart = chart.properties(
    width=700,
    height=350,
)

chart




/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)

In [75]:
bars

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [74]:
df

,country,date,value,iso3,flag,rank
0,Albania,2014,0.013465,ALB,🇦🇱,16
1,Albania,2015,0.011623,ALB,🇦🇱,16
2,Albania,2016,0.011036,ALB,🇦🇱,16
3,Albania,2017,0.011091,ALB,🇦🇱,16
4,Albania,2018,0.011605,ALB,🇦🇱,16
...,...,...,...,...,...,...
295,United States,2019,0.035078,USA,🇺🇸,1
296,United States,2020,0.036352,USA,🇺🇸,1
297,United States,2021,0.034769,USA,🇺🇸,1
298,United States,2022,0.034496,USA,🇺🇸,1


In [72]:
df = full_df.copy()
df[df['date'] == 2023].sort_values('value', ascending=False).reset_index(drop=True).reset_index()

,index,country,date,value,iso3,flag
0,0,Poland,2023,0.039011,POL,🇵🇱
1,1,United States,2023,0.034929,USA,🇺🇸
2,2,Greece,2023,0.030074,GRC,🇬🇷
3,3,Estonia,2023,0.027266,EST,🇪🇪
4,4,Lithuania,2023,0.025437,LTU,🇱🇹
5,5,Finland,2023,0.024497,FIN,🇫🇮
6,6,Romania,2023,0.024406,ROU,🇷🇴
7,7,Hungary,2023,0.024268,HUN,🇭🇺
8,8,Latvia,2023,0.022660,LVA,🇱🇻
9,9,United Kingdom,2023,0.020666,GBR,🇬🇧


In [64]:
df.sort_values('rank').drop_duplicates('iso3')

,country,date,value,iso3,flag,rank
0,Albania,2014,0.013465,ALB,🇦🇱,270
19,Belgium,2023,0.011338,BEL,🇧🇪,271
29,Bulgaria,2023,0.018408,BGR,🇧🇬,272
36,Canada,2020,0.014160,CAN,🇨🇦,273
40,Croatia,2014,0.018203,HRV,🇭🇷,274
55,Czechia,2019,0.011801,CZE,🇨🇿,275
69,Denmark,2023,0.016467,DNK,🇩🇰,276
76,Estonia,2020,0.022983,EST,🇪🇪,277
88,Finland,2022,0.016807,FIN,🇫🇮,278
99,France,2023,0.019044,FRA,🇫🇷,279


In [55]:
df

,date,value,label
0,2012-04-01,170012,
1,2012-07-01,168182,
2,2012-10-01,165894,
3,2013-01-01,162250,
4,2013-04-01,160712,
5,2013-07-01,158185,
6,2013-10-01,156694,
7,2014-01-01,152438,
8,2014-04-01,150891,
9,2014-07-01,148709,


In [47]:
df

,date,value,label
0,2012-04-01,170012,
1,2012-07-01,168182,
2,2012-10-01,165894,
3,2013-01-01,162250,
4,2013-04-01,160712,
5,2013-07-01,158185,
6,2013-10-01,156694,
7,2014-01-01,152438,
8,2014-04-01,150891,
9,2014-07-01,148709,


In [ ]:
df['annual_change'] = 

In [7]:
line

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [6]:
df

,date,value,label
0,2012-04-01,170 012,Armed Forces Personnel
1,2012-07-01,168 182,
2,2012-10-01,165 894,
3,2013-01-01,162 250,
4,2013-04-01,160 712,
5,2013-07-01,158 185,
6,2013-10-01,156 694,
7,2014-01-01,152 438,
8,2014-04-01,150 891,
9,2014-07-01,148 709,
